In [19]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from dotenv import load_dotenv
import os

In [20]:
load_dotenv()

True

In [21]:
api_key = os.getenv("OPENAI_API_KEY")

In [22]:
required_exts = ['.pdf', '.tex']
reader = SimpleDirectoryReader(input_dir = "../data", required_exts = required_exts)

In [23]:
docs = reader.load_data()

2026-02-12 11:44:22,625 - WARNING - Ignoring wrong pointing object 173 0 (offset 0)
2026-02-12 11:44:22,629 - WARNING - Ignoring wrong pointing object 376 0 (offset 0)
2026-02-12 11:44:22,630 - WARNING - Ignoring wrong pointing object 393 0 (offset 0)
2026-02-12 11:44:22,631 - WARNING - Ignoring wrong pointing object 425 0 (offset 0)
2026-02-12 11:44:22,631 - WARNING - Ignoring wrong pointing object 427 0 (offset 0)
2026-02-12 11:44:22,631 - WARNING - Ignoring wrong pointing object 434 0 (offset 0)
2026-02-12 11:44:22,634 - WARNING - Ignoring wrong pointing object 652 0 (offset 0)
2026-02-12 11:44:22,635 - WARNING - Ignoring wrong pointing object 678 0 (offset 0)
2026-02-12 11:44:22,637 - WARNING - Ignoring wrong pointing object 781 0 (offset 0)
2026-02-12 11:44:22,638 - WARNING - Ignoring wrong pointing object 837 0 (offset 0)
2026-02-12 11:44:22,638 - WARNING - Ignoring wrong pointing object 840 0 (offset 0)
2026-02-12 11:44:22,638 - WARNING - Ignoring wrong pointing object 843 0 (of

In [24]:
print(f"Loaded {len(docs)} documents")

Loaded 1462 documents


In [25]:
# See what one chunk looks like
print(f"Chunk length: {len(docs[0].text)} characters")
print(f"Preview: {docs[0].text[:500]}")

Chunk length: 147 characters
Preview: Gareth James • Daniela Witten •
Trevor Hastie • Robert Tibshirani
An Introduction to Statistical
Learning
with Applications in R
Second Edition
123


In [27]:
index = VectorStoreIndex.from_documents(docs)

2026-02-12 11:52:10,908 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:11,907 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:12,695 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:15,771 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:16,412 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:17,125 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:17,901 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:18,792 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:19,528 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-12 11:52:20,143 - INFO - HTTP